In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np

# Directories for each class of images
class_dirs = ["Bacterial Blight", "Downy Mildew", "Blast", "Dead Heart", "Healthy"]

# Data directory where the images are stored
image_dir = "dataset"

# Function to preprocess images by converting them to HSV
def preprocess_to_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_bound = np.array([25, 40, 40])   
    upper_bound = np.array([85, 255, 255])
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    masked_image = cv2.bitwise_and(hsv_image, hsv_image, mask=mask)
    
    return hsv_image

# Custom ImageDataGenerator with the HSV preprocessing
def hsv_image_generator(generator, directory, subset):
    data_gen = generator.flow_from_directory(
        directory,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        subset=subset
    )
    
    while True:
        batch_x, batch_y = next(data_gen)
        batch_x_hsv = np.array([preprocess_to_hsv(img) for img in batch_x])
        yield batch_x_hsv, batch_y

# ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,          
    rotation_range=30,       
    width_shift_range=0.2,   
    height_shift_range=0.2,  
    shear_range=0.2,         
    zoom_range=0.2,          
    horizontal_flip=True,    
    vertical_flip=True,      
    validation_split=0.2     
)

# Create generators for training and validation
train_generator = hsv_image_generator(train_datagen, image_dir, subset='training')

validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
validation_generator = hsv_image_generator(validation_datagen, image_dir, subset='validation')

# Function to count the number of images in each subset
# Function to count the number of images in the dataset
def count_images(directory):
    total = 0
    for class_dir in os.listdir(directory):
        class_path = os.path.join(directory, class_dir)
        if os.path.isdir(class_path):  # Ensure it's a folder
            total += len(os.listdir(class_path))  # Count images in the folder
    return total

# Manually count the number of images in the dataset
train_image_count = count_images(image_dir)  # No 'training' subfolder
validation_image_count = int(train_image_count * 0.2)  # 20% validation split

# Batch size used for training
batch_size = 32

# Calculate steps per epoch and validation steps
steps_per_epoch = int(np.ceil(train_image_count / batch_size))
validation_steps = int(np.ceil(validation_image_count / batch_size))


# Define the CNN model with batch normalization and dropout for regularization
def build_enhanced_cnn_model(input_shape=(256, 256, 3), num_classes=5):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build the enhanced CNN model
cnn_model = build_enhanced_cnn_model(input_shape=(256, 256, 3), num_classes=len(class_dirs))

# Print model summary
cnn_model.summary()

# Define callbacks for learning rate adjustment and early stopping
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
learning_rate_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model using the generators
history = cnn_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, learning_rate_scheduler]
)

# Save the model
cnn_model.save("rice_leaf_disease_classifier_hsv_refactored.h5")

# Evaluate the model on the validation data
val_loss, val_acc = cnn_model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 254, 254, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 125, 125, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 60, 60, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │      12,845,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,237,957 (50.50 MB)

 Trainable params: 13,236,485 (50.49 MB)

 Non-trainable params: 1,472 (5.75 KB)

Found 4723 images belonging to 5 classes.
Epoch 1/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3855 - loss: 2.0772Found 1178 images belonging to 5 classes.
185/185 ━━━━━━━━━━━━━━━━━━━━ 681s 4s/step - accuracy: 0.3858 - loss: 2.0754 - val_accuracy: 0.3404 - val_loss: 1.9436 - learning_rate: 0.0010
Epoch 2/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 757s 4s/step - accuracy: 0.4961 - loss: 1.4185 - val_accuracy: 0.4847 - val_loss: 1.9754 - learning_rate: 0.0010
Epoch 3/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 812s 4s/step - accuracy: 0.5340 - loss: 1.2421 - val_accuracy: 0.3217 - val_loss: 4.3576 - learning_rate: 0.0010
Epoch 4/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 798s 4s/step - accuracy: 0.6093 - loss: 1.0373 - val_accuracy: 0.6095 - val_loss: 1.0189 - learning_rate: 0.0010
Epoch 5/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 763s 4s/step - accuracy: 0.6625 - loss: 0.9261 - val_accuracy: 0.3141 - val_loss: 2.4179 - learning_rate: 0.0010
Epoch 6/20
185/185 ━━━━━━━━━━━━━━━━━━━━ 934s 5s/step - accuracy: 0.6546 - loss